In [4]:
#词典库  vocab.txt 为词库
vocab = {line.rstrip() for line in open('./data/vocab.txt',encoding='utf-8')}

In [61]:
#生成编辑距离为2的候选集合
def generate_candidates_two(word_one):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    candidates_two = []
    for word in word_one:
        splits = [(word[:i] , word[i:]) for i in range(len(word) + 1)]
        #insert操作
        inserts = [L+c+R for L,R in splits for c in letters]
        #repalce操作
        replaces = [L+c+R[1:] for L,R in splits if R for c in letters]
        #delete操作
        deletes = [L+R[1:] for L,R in splits if R]
    
        candidates_two += set(inserts + replaces + deletes)
    return [word for word in candidates_two if word in vocab]

In [68]:
#需要生成所有的候选集合
def generate_candidates(word):
    """
    word:给定的输入(错误的输入)
    返回所有(valid)候选集合
    """
    #生成编辑距离为1的单词
    # 1.insert 2.delete 3.replace
    #假设使用26个字符
    letters = 'abcdefghijklmnopqrstuvwxyz'
    
    splits = [(word[:i] , word[i:]) for i in range(len(word) + 1)]
    #insert操作
    inserts = [L+c+R for L,R in splits for c in letters]
    #repalce操作
    replaces = [L+c+R[1:] for L,R in splits if R for c in letters]
    #delete操作
    deletes = [L+R[1:] for L,R in splits if R]
    
    candidate_one = set(inserts + replaces + deletes)
    #过滤掉不存在词典库中的单词
    return [word for word in candidate_one if word in vocab]
#     word_one =  [word for word in candidate_one if word in vocab]
#     if len(word_one) < 1:
#         return generate_candidates_two(candidate_one)
#     else:
#         return [word for word in candidate_one if word in vocab]
#generate_candidates("appc")

In [69]:
from nltk.corpus import reuters #路透社语料库
#读取语料库
categories = reuters.categories()
corpus = reuters.sents(categories = categories)

In [70]:
# 构建语言模型 unigram bigram trigram
term_count = {} #单个单词出现的次数
bigram_count = {} #连续两个单词出现的次数
for doc in corpus:
    doc = ['<s>'] + doc
    for i in range(0,len(doc) - 1):
        #bigram: [i, i+1]
        term = doc[i]
        bigram = doc[i:i+2]
        
        if term in term_count:
            term_count[term] += 1
        else:
            term_count[term] = 1
        bigram = ' '.join(bigram)
        if bigram in bigram_count:
            bigram_count[bigram] += 1
        else:
            bigram_count[bigram] = 1
# sklearn里面有现成的包

In [71]:
#用户打错的概率统计 - channel probability    
channel_prob = {} #就是一个二维元组，第一维是正确单词，第二维是错误单词
for line in open("./data/spell-errors.txt"):
    items = line.split(":")
    correct = items[0].strip()
    mistakes = [word.strip() for word in items[1].strip().split(",")]
    channel_prob[correct] = {}
    for mis in mistakes:
        channel_prob[correct][mis] = 1.0 / len(mistakes)

In [82]:
import numpy as np
V = len(term_count.keys()) #smothing操作中,单词的总数V
file = open("./data/testdata.txt",'r')

for line in file:
    items = line.rstrip().split('\t') # items 是一个三维数组， items[2]是一个句子
    line = items[2].split()
    line = ['<s>'] + line  #把获取的一句话，进行与计算bigram_count相同的预处理
    for word in line:
        if word not in vocab: 
            #如果句子中，有单词不在词典中    word:错误的单词
            #step1：生成所有的(valid)候选集合
            candidates = generate_candidates(word)
            if len(candidates) < 1:
                continue
            probs = []
            #对于每一个candidate，计算它的score
            # score = p(correct) * p(mistake|correct)
            #       = log p(correct) + log p(mistake|correct)
            #返回score最大的candidate
            for candi in candidates:
                prob = 0
                #计算 channel概率  出错的概率   
                if candi in channel_prob and word in channel_prob[candi]:#正确单词在channel_prob中，并且也存在对应错误写法
                    prob += np.log(channel_prob[candi][word])
                else:
                    prob += np.log(0.00001)
                # 计算语言模型的概率
                idx = line.index(word) #找到错误单词，在句子中的位置
                #计算idx位置左边的bigram_prob
                left_bigram = line[idx-1] +' ' + candi
                if left_bigram in bigram_count:#判断bigram 是否在bigtam_count中
                    prob += np.log(bigram_count[left_bigram] + 1.0 / (term_count[candi] + V))
                else:
                    prob += np.log(1.0 / V)
                #计算idx位置右边的bigram_prob
                if idx != len(line)-1:
                    right_bigram = candi + ' '+ line[idx+1]
                    if right_bigram in bigram_count:
                        prob += np.log(bigram_count[right_bigram] + 1.0 /(term_count[candi] + V))
                probs.append(prob)
            max_index = probs.index(max(probs))
            #print(word,candidates[max_index])

In [12]:
items = 'hello world'
line = items.strip().split()
print(items[1])
line = items.rstrip()
print(line[1])

e
e
